In [37]:
from pymilvus import MilvusClient, DataType

client = MilvusClient(
    uri="http://localhost:19530",
    token="root:Milvus"
)

res = client.list_collections()

print("collections are: ", res)



collections are:  ['_emails_attachments']


In [39]:
import json

res = client.describe_collection(
    collection_name="_emails_attachments"
)

print(json.dumps(res, indent=4))


{
    "collection_name": "_emails_attachments",
    "auto_id": false,
    "num_shards": 1,
    "description": "",
    "fields": [
        {
            "field_id": 100,
            "name": "id",
            "description": "",
            "type": 21,
            "params": {
                "max_length": 65535
            },
            "is_primary": true
        },
        {
            "field_id": 101,
            "name": "embedding",
            "description": "",
            "type": 101,
            "params": {
                "dim": 1536
            }
        }
    ],
    "functions": [],
    "aliases": [],
    "collection_id": 456790168969799483,
    "consistency_level": 1,
    "properties": {},
    "num_partitions": 1,
    "enable_dynamic_field": true,
    "created_timestamp": 456887103297945609
}


In [2]:
from utils.config_loader import app_config
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, StorageContext
from llama_index.vector_stores.milvus import MilvusVectorStore

In [3]:
# 1. Connect to Milvus
milvus_store = MilvusVectorStore(
    uri=app_config.MILVUS_URL,
    dim=app_config.MILVUS_DIM,
    overwrite= False,
    collection_name="_emails_attachments"
)

2025-03-25 15:30:54,865 [DEBUG][_create_connection]: Created new connection using: 786593bf3c994c6f926a177dbc05a73e (async_milvus_client.py:600)


### Example 1: Using LlamaIndex to query the vector store

In [4]:
## Step 1: Prepare global settings

# Access appplication environment variables
model = app_config.LLAMAINDEX_MODEL
embed_model= app_config.LLAMAINDEX_EMBEDDING

# define global Setting that sets the LLM and ebemdding model.
# There is no need to pass in any llm or embedding model to the LlamaIndexTool query engine.
# The LlamaIndexTool will use the global Settings to get the llm and embedding model.
Settings.llm = OpenAI(temperature=0.1, model=model)
Settings.embed_model = OpenAIEmbedding(model=embed_model)

In [5]:
# Step 2: Load index from existing Milvus vector store
index = VectorStoreIndex.from_vector_store(
    vector_store=milvus_store
)

In [6]:
# Step 3. Create the query and chat engine using the LLM
query_engine = index.as_query_engine()  # no parameter needed, taken care off by the global Settings
chat_engine = index.as_chat_engine()


In [8]:

# Step 4A. Run your query
response = query_engine.query("summarise the packinglist for order for customer ' PT Interindo Kharisma ' ")
print(response)

The packing list for the order for the customer 'PT Interindo Kharisma' includes the following details: The destination terminal is associated with the booking reference '726'. The ready date and stuffing date are not specified in the provided information.


In [7]:
response = query_engine.query("what are the meta-data for shipID?")
print(response)

The meta-data for shipID includes "Qty/20'", "Vessel", "VOYAGE", "ETA SIN", "ETA UTC1", "Destination Terminal", "Booking Ref", "Ready Date", and "Stuffing Date".


In [8]:
# Step 4B. Run your  cchat
response = chat_engine.chat("who is the owner fro shipment 2230500347?")
print(response)

The owner of shipment 2230500347 is UBTS TECHNOLOGIES PTE LTD.


In [9]:
# Step 4B. Run your  cchat
response = chat_engine.chat("what is inside of it?")
print(response)

The shipment with reference number 2230500347 contains 1 package with a weight of 5131.860 kg and a volume of 50.000 m³. It is transported in a 40 GP container with the seal number PM00113420.


In [ ]:
from pymilvus import Collection
from pymilvus import connections



# Connect to Milvus first (adjust host and port as needed)
collection = Collection("_emails_attachments")

# Query all metadata entries (limit to a few for performance)
results = collection.query(
    expr=None,  # No filtering, get all rows
    output_fields=["id", "$meta"],
    limit=10
)

# Print results
for r in results:
    print(r["id"], r["$meta"])


ConnectionNotExistException: <ConnectionNotExistException: (code=1, message=should create connection first.)>